# Advanced Person Random Walk Publisher with Weather Awareness

This notebook creates a named person who:
- Performs a random walk starting from City Hall
- Subscribes to weather updates via MQTT
- Moves to the nearest coffee shop when it rains
- Resumes random walk when weather clears
- Publishes location updates (with color) once per second

**Usage:**
1. Edit the `PERSON_NAME` and `COLOR` in Cell 2
2. Run all cells to start the walker
3. Launch `weather_controller.ipynb` to generate weather changes
4. Launch `map_viewer_advanced.ipynb` to visualize on map
5. Watch as this person reacts to weather and seeks shelter!

In [ ]:
import asyncio
import json
import math
import random
import time
from typing import Tuple, List, Dict, Any, Optional

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher

In [ ]:
# ===== CONFIGURE YOUR PERSON HERE =====
PERSON_NAME = "Alice"  # Change this for each notebook!
COLOR = "#b40d29"       # Hex color code (red). Change for different colors.
# ======================================

In [ ]:
# City Hall coordinates (Copenhagen)
CITY_HALL_LNGLAT = (12.5683, 55.6761)

# Random walk parameters
STEP_M = 6.0              # meters per step
STEP_S = 0.1              # seconds between steps
MAX_RADIUS_M = 250.0      # max distance from City Hall
CAFE_APPROACH_DISTANCE_M = 20.0  # stop moving when this close to cafe
SEED = int(time.time())   # random seed based on current time

# MQTT setup
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix=f"walker-advanced-{PERSON_NAME}")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

publisher = MqttPublisher(connector)
print(f"✓ Connected to MQTT broker as {PERSON_NAME}")

In [ ]:
# ===== HELPER FUNCTIONS =====

LngLat = Tuple[float, float]

def haversine_m(a: LngLat, b: LngLat) -> float:
    """Great-circle distance in meters between two (lng, lat) points."""
    lng1, lat1 = a
    lng2, lat2 = b
    r = 6_371_000.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = phi2 - phi1
    dlambda = math.radians(lng2 - lng1)
    h = (math.sin(dphi / 2) ** 2) + math.cos(phi1) * math.cos(phi2) * (math.sin(dlambda / 2) ** 2)
    return 2 * r * math.asin(math.sqrt(h))

def find_nearest_cafe(current_pos: LngLat, cafes: List[Dict]) -> Optional[Dict]:
    """Find the nearest cafe to current position."""
    if not cafes:
        return None
    best_cafe = None
    best_distance = float("inf")
    for cafe in cafes:
        cafe_pos = (cafe["lng"], cafe["lat"])
        distance = haversine_m(current_pos, cafe_pos)
        if distance < best_distance:
            best_distance = distance
            best_cafe = cafe
    return best_cafe

def generate_waypoints(start: LngLat, end: LngLat, num_waypoints: int = 10) -> List[LngLat]:
    """
    Generate waypoints along a straight line from start to end.
    Useful for smooth navigation toward a target.
    """
    lng1, lat1 = start
    lng2, lat2 = end
    
    waypoints = []
    for i in range(1, num_waypoints + 1):
        t = i / (num_waypoints + 1)
        lng = lng1 + t * (lng2 - lng1)
        lat = lat1 + t * (lat2 - lat1)
        waypoints.append((lng, lat))
    waypoints.append(end)  # Always end at destination
    return waypoints

print("✓ Helper functions defined")

In [ ]:
# State management
state = {
    "weather": "sunny",
    "cafes": [],
    "is_seeking_shelter": False,
    "target_cafe": None,
    "waypoints": [],
    "waypoint_index": 0,
}

def on_weather_message(client, userdata, message):
    """Handle incoming weather updates."""
    try:
        data = json.loads(message.payload.decode())
        state["weather"] = data["state"]
        print(f"  [{PERSON_NAME}] Weather update: {state['weather']}")
    except Exception as e:
        print(f"  [{PERSON_NAME}] Error parsing weather: {e}")

def on_cafes_message(client, userdata, message):
    """Handle incoming cafe location updates."""
    try:
        data = json.loads(message.payload.decode())
        state["cafes"] = data.get("cafes", [])
        if not any(state["cafes"]):
            print(f"  [{PERSON_NAME}] No cafes available")
    except Exception as e:
        print(f"  [{PERSON_NAME}] Error parsing cafes: {e}")

# Set up subscriptions
def setup_subscriptions():
    # Create a dispatcher to handle multiple topics
    def on_message_dispatcher(client, userdata, message):
        if message.topic == "weather/status":
            on_weather_message(client, userdata, message)
        elif message.topic == "cafes/locations":
            on_cafes_message(client, userdata, message)
    
    connector.client.on_message = on_message_dispatcher
    connector.client.subscribe("weather/status", qos=0)
    connector.client.subscribe("cafes/locations", qos=0)

setup_subscriptions()
print(f"✓ [{PERSON_NAME}] Subscribed to weather/status and cafes/locations")

In [ ]:
async def advanced_random_walk_publisher(
    person_name: str,
    color: str,
    *,
    seed: int = 42,
    step_m: float = 6.0,
    step_s: float = 1.0,
    max_radius_m: float = 250.0,
) -> None:
    """
    Perform a weather-aware random walk.
    
    When sunny: normal random walk around City Hall
    When raining: navigate to nearest cafe and stay there
    
    Publishes to topic: persons/{person_name}/location
    Message format: {"lng": float, "lat": float, "color": str, "name": str, "timestamp": float}
    """
    rng = random.Random(seed)
    center_lng, center_lat = CITY_HALL_LNGLAT
    meters_per_deg_lat = 111_320.0
    meters_per_deg_lng = 111_320.0 * math.cos(math.radians(center_lat))
    
    # Start at City Hall
    x_m = 0.0
    y_m = 0.0
    
    topic = f"persons/{person_name}/location"
    print(f"\n[{person_name}] Starting advanced random walk (color: {color})")
    print(f"[{person_name}] Publishing to topic: {topic}")
    print(f"[{person_name}] I will seek shelter when it rains!\n")
    
    step_count = 0
    
    while True:
        current_weather = state["weather"]
        
        if current_weather == "rain" and not state["is_seeking_shelter"]:
            # Start seeking shelter
            print(f"[{person_name}] 🌧️  It's raining! Looking for shelter...")
            current_pos = (center_lng + (x_m / meters_per_deg_lng), center_lat + (y_m / meters_per_deg_lat))
            nearest_cafe = find_nearest_cafe(current_pos, state["cafes"])
            
            if nearest_cafe:
                state["is_seeking_shelter"] = True
                state["target_cafe"] = nearest_cafe
                cafe_pos = (nearest_cafe["lng"], nearest_cafe["lat"])
                distance = haversine_m(current_pos, cafe_pos)
                print(f"[{person_name}] 🏃 Running to {nearest_cafe['name']} ({distance:.1f}m away)")
                # Generate waypoints for smooth approach
                state["waypoints"] = generate_waypoints(current_pos, cafe_pos, num_waypoints=int(distance / 5))
                state["waypoint_index"] = 0
        
        elif current_weather == "sunny" and state["is_seeking_shelter"]:
            # Stop seeking shelter
            print(f"[{person_name}] ☀️  The rain stopped! Resuming my walk...")
            state["is_seeking_shelter"] = False
            state["target_cafe"] = None
            state["waypoints"] = []
        
        # Decide next position
        if state["is_seeking_shelter"] and state["waypoints"]:
            # Move toward cafe along waypoints
            if state["waypoint_index"] < len(state["waypoints"]):
                next_pos = state["waypoints"][state["waypoint_index"]]
                x_m = (next_pos[0] - center_lng) * meters_per_deg_lng
                y_m = (next_pos[1] - center_lat) * meters_per_deg_lat
                state["waypoint_index"] += 1
            else:
                # Reached cafe, stay here
                cafe_pos = (state["target_cafe"]["lng"], state["target_cafe"]["lat"])
                x_m = (cafe_pos[0] - center_lng) * meters_per_deg_lng
                y_m = (cafe_pos[1] - center_lat) * meters_per_deg_lat
        else:
            # Normal random walk
            theta = rng.random() * 2.0 * math.pi
            x_m += step_m * math.cos(theta)
            y_m += step_m * math.sin(theta)
            
            # Keep within max radius (soft boundary)
            r = math.hypot(x_m, y_m)
            if r > max_radius_m:
                scale = max_radius_m / r
                x_m *= scale
                y_m *= scale
        
        # Convert to lng/lat
        lng = center_lng + (x_m / meters_per_deg_lng)
        lat = center_lat + (y_m / meters_per_deg_lat)
        
        # Create message
        message = {
            "lng": lng,
            "lat": lat,
            "color": color,
            "name": person_name,
            "timestamp": time.time(),
        }
        
        # Publish to MQTT
        publisher.publish_json(topic, json.dumps(message), qos=0)
        
        step_count += 1
        if step_count % 20 == 0:
            status = "sheltering" if state["is_seeking_shelter"] else "walking"
            print(f"[{person_name}] {step_count:4d} steps ({status}), at ({lng:.6f}, {lat:.6f})")
        
        await asyncio.sleep(step_s)

In [ ]:
# Start the advanced random walk in the background
task = asyncio.create_task(
    advanced_random_walk_publisher(
        PERSON_NAME,
        COLOR,
        seed=SEED,
        step_m=STEP_M,
        step_s=STEP_S,
        max_radius_m=MAX_RADIUS_M,
    )
)
print(f"\n✓ {PERSON_NAME} is walking and watching the weather!")
print("Run the next cell to stop.")

In [ ]:
# Stop the walker
task.cancel()
connector.disconnect()
print(f"✓ {PERSON_NAME} stopped walking.")